In [ ]:
!pip install catboost
!pip install rfpimp
!pip install reg_resampler

     |████████████████████████████████| 65.9MB 42kB/s 
     |████████████████████████████████| 163kB 19.7MB/s 
  Created wheel for rfpimp: filename=rfpimp-1.3.5-cp36-none-any.whl size=10260 sha256=7e7d9dc49fd1cf9b6a60fbd0395094bfc2a34ba4c7fd7038eef1b521638a4f88
  Stored in directory: /root/.cache/pip/wheels/2a/c8/bf/80f91224be00870f84a85aa1782cf6cd62c1289a173b66b8e9
  Created wheel for stratx: filename=stratx-0.4.1-cp36-none-any.whl size=35079 sha256=d26df7d8ed5db16fce44f40f1d42831716a45fd9aad44d2c0c6caea2ccc9b006
  Stored in directory: /root/.cache/pip/wheels/ca/c1/e1/3e4c41e3420fe9c706dd4d8ef8d7fb5aba1b6a9547649a2d00
Successfully built rfpimp stratx


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from scipy import stats
from sklearn.model_selection import KFold,train_test_split,GridSearchCV
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from rfpimp import *
from catboost import CatBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor

from imblearn.over_sampling import SMOTE,RandomOverSampler
from reg_resampler import resampler
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


## Here is the work of Krithik , Devrup ,Sayantan

In [2]:
ETCCOLUMNS=['size_npear / damage_size', 'PERCENTILE(damage_size) / damage_size',
       'PERCENTILE(mass_npea) / PERCENTILE(exposed_area)',
       'malign_penalty / damage_size', 'PERCENTILE(exposed_area) / size_npear',
       'damage_ratio * err_malign',
       'malign_penalty / err_malign * malign_ratio / damage_size',
       'damage_size / size_npear',
       'malign_penalty / damage_size * malign_ratio / damage_size',
       'malign_ratio / exposed_area * PERCENTILE(err_malign)',
       'malign_penalty / PERCENTILE(malign_penalty)',
       'damage_ratio / exposed_area * damage_size / malign_ratio',
       'PERCENTILE(malign_penalty) / damage_size',
       'malign_penalty / mass_npea * PERCENTILE(damage_ratio)',
       'err_malign / mass_npea * PERCENTILE(std_dev_malign)',
       'err_malign / damage_size * PERCENTILE(damage_ratio)',
       'damage_ratio * PERCENTILE(std_dev_malign)',
       'PERCENTILE(mass_npea) / PERCENTILE(malign_penalty)',
       'malign_ratio / err_malign * PERCENTILE(malign_penalty)',
       'PERCENTILE(std_dev_malign) / mass_npea',
       'malign_penalty / std_dev_malign * malign_ratio / exposed_area',
       'damage_size / mass_npea * std_dev_malign / mass_npea',
       'PERCENTILE(damage_ratio) * PERCENTILE(err_malign)',
       'PERCENTILE(err_malign) / PERCENTILE(exposed_area)',
       'damage_ratio * PERCENTILE(err_malign)',
       'damage_ratio / damage_size * malign_penalty / damage_size',
       'PERCENTILE(err_malign) / std_dev_malign',
       'damage_ratio / std_dev_malign * damage_size',
       'PERCENTILE(malign_penalty) / malign_ratio',
       'PERCENTILE(std_dev_malign) / err_malign',
       'damage_ratio * PERCENTILE(damage_size)',
       'err_malign / std_dev_malign * malign_ratio / mass_npea',
       'PERCENTILE(malign_penalty) / err_malign',
       'malign_penalty / size_npear * PERCENTILE(damage_ratio)',
       'err_malign / size_npear * malign_ratio / damage_ratio',
       'err_malign / damage_size * malign_penalty / damage_size']
df_train = pd.read_csv("/content/drive/My Drive/Colab Notebooks/MachineHack/Melanoma_Tumor_Size_Prediction/Train.csv")
df_test = pd.read_csv("/content/drive/My Drive/Colab Notebooks/MachineHack/Melanoma_Tumor_Size_Prediction/Test.csv")
sub = pd.read_csv("/content/drive/My Drive/Colab Notebooks/MachineHack/Melanoma_Tumor_Size_Prediction/sample_submission.csv")
#getting selected features
train_list = [y for x in [ETCCOLUMNS, df_test.columns.to_list()] for y in x] 
test_list = [y for x in [ETCCOLUMNS, df_test.columns.to_list()] for y in x] 

In [3]:
TRAIN = pd.read_csv("/content/drive/My Drive/Colab Notebooks/MachineHack/Melanoma_Tumor_Size_Prediction/TRAIN.csv")
TEST = pd.read_csv("/content/drive/My Drive/Colab Notebooks/MachineHack/Melanoma_Tumor_Size_Prediction/TEST.csv")
Y_TRAIN = pd.read_csv("/content/drive/My Drive/Colab Notebooks/MachineHack/Melanoma_Tumor_Size_Prediction/Y.csv")


## Using Feature Tools create Features and Here are the selected best features from it

In [4]:
TRAIN

,mass_npea,size_npear,malign_ratio,damage_size,err_malign,malign_penalty,damage_ratio,key,PERCENTILE(mass_npea),PERCENTILE(size_npear),PERCENTILE(damage_size),PERCENTILE(err_malign),PERCENTILE(malign_penalty),malign_penalty / damage_size,size_npear / malign_penalty,damage_size / mass_npea,damage_ratio / mass_npea,size_npear / damage_ratio,damage_size / std_dev_malign,malign_penalty / exposed_area,err_malign / exposed_area,damage_ratio / err_malign,err_malign / malign_ratio,exposed_area / std_dev_malign,exposed_area / mass_npea,malign_ratio / mass_npea,mass_npea / malign_penalty,size_npear / err_malign,mass_npea / damage_size,malign_penalty / malign_ratio,malign_ratio / damage_size,damage_ratio / malign_ratio,damage_ratio / damage_size,malign_penalty / std_dev_malign,err_malign / std_dev_malign,size_npear / damage_size,exposed_area / size_npear,malign_ratio / damage_ratio,std_dev_malign / size_npear,damage_size / malign_ratio,...,PERCENTILE(malign_penalty) * PERCENTILE(malign_ratio),malign_ratio / damage_size * std_dev_malign / damage_ratio,err_malign / damage_size * PERCENTILE(size_npear),damage_size * PERCENTILE(damage_ratio),damage_ratio * PERCENTILE(size_npear),damage_size / std_dev_malign * std_dev_malign / damage_size,exposed_area / malign_penalty * malign_penalty / exposed_area,damage_ratio * PERCENTILE(std_dev_malign),err_malign / size_npear * PERCENTILE(malign_penalty),PERCENTILE(damage_ratio) * PERCENTILE(malign_penalty),malign_penalty / damage_size * malign_ratio / damage_size,damage_size / damage_ratio * malign_ratio / std_dev_malign,exposed_area / mass_npea * mass_npea / exposed_area,damage_size / mass_npea * mass_npea / damage_size,PERCENTILE(damage_ratio) * PERCENTILE(exposed_area),malign_penalty / size_npear * PERCENTILE(damage_ratio),PERCENTILE(err_malign) * size_npear / damage_size,malign_ratio / err_malign * PERCENTILE(err_malign),damage_ratio / std_dev_malign * PERCENTILE(err_malign),damage_size / std_dev_malign * size_npear / std_dev_malign,mass_npea / std_dev_malign * size_npear / std_dev_malign,malign_ratio / malign_penalty * PERCENTILE(malign_penalty),damage_ratio / malign_ratio * PERCENTILE(size_npear),damage_ratio / err_malign * PERCENTILE(err_malign),err_malign / malign_penalty * PERCENTILE(malign_penalty),err_malign / damage_size * malign_ratio / damage_size,err_malign / mass_npea * mass_npea / err_malign,malign_ratio / err_malign * PERCENTILE(malign_penalty),malign_penalty / damage_size * PERCENTILE(malign_ratio),damage_ratio / malign_penalty * PERCENTILE(malign_penalty),PERCENTILE(damage_ratio) * PERCENTILE(damage_size),exposed_area / size_npear * size_npear / exposed_area,damage_ratio / std_dev_malign * PERCENTILE(damage_size),malign_ratio / err_malign * PERCENTILE(damage_size),err_malign / size_npear * PERCENTILE(std_dev_malign),malign_penalty / size_npear * size_npear / malign_penalty,damage_ratio / err_malign * exposed_area / err_malign,damage_ratio / damage_size * malign_penalty / damage_size,malign_penalty / mass_npea * mass_npea / malign_penalty,PERCENTILE(damage_ratio) * PERCENTILE(size_npear)
0,6930.90,2919.02,0.42116,51.8298,2758.76,72,39.3620,6930.9_2919.02_0.42116_51.8298_988829.407_109....,0.248737,0.571165,0.122388,0.137121,0.638412,1.389162,40.541944,0.007478,0.005679,74.158325,0.473388,0.000073,0.002790,0.014268,6550.384652,9031.477774,142.669698,0.000061,96.262500,1.058091,133.724228,170.956406,0.008126,93.460917,0.759447,0.657612,25.197147,56.319338,338.753899,0.010700,0.037508,123.064394,...,0.615060,0.022602,30.401550,40.630637,22.482184,1.0,1.0,14.786922,0.603362,0.500467,0.011288,0.005065,1.0,1.0,0.218876,0.019336,7.722584,0.000021,0.049297,12.620934,1687.724642,0.003734,53.381575,0.001956,24.461474,0.432516,1.0,0.000097,1.338349,0.349016,0.095943,1.0,0.044000,0.000019,0.355040,1.0,5.114117,1.054995,1.0,0.447750
1,15635.70,4879.36,0.31206,223.5500,5952.53,240,22.0253,15635.7_4879.36_0.31206_223.55_2058426.3009_24...,0.903163,0.881797,0.963655,0.961764,0.979435,1.073585,20.3306

## Noting Key to Handle A Small Leak(600 samples are common in both test and train)

In [5]:
# Devrup's
key=df_train.drop(['tumor_size'],axis=1).columns.tolist()
key2=['mass_npea', 'size_npear', 'malign_ratio', 'damage_size',
       'exposed_area', 'std_dev_malign', 'err_malign', 'malign_penalty']

df_test['key']=df_test[key].apply(lambda x: '_'.join(x.astype('str')), axis = 1)
df_train['key']=df_train[key].apply(lambda x: '_'.join(x.astype('str')), axis = 1)

# df_test['key2']=df_test[key2].apply(lambda x: '_'.join(x.astype('str')), axis = 1)
# df_train['key2']=df_train[key2].apply(lambda x: '_'.join(x.astype('str')), axis = 1)
# df2=pd.DataFrame(df_train.groupby(['key2'],as_index=False)['tumor_size'].mean())
# df2=df2.reset_index()
# encode2 = dict(zip(df2.key2, df2.tumor_size))

df=pd.DataFrame(df_train.groupby(['key'],as_index=False)['tumor_size'].mean())
df=df.reset_index()
encode = dict(zip(df.key, df.tumor_size))



df_test['tumor_size']=df_test.key.map(encode)
#############key2 on of#################################################
# df_test['tumor_size']=df_test['tumor_size'].fillna(df_test['key2'].map(encode2))
##############################################################################
df_test2=df_test[np.isnan(df_test.tumor_size)].drop(['tumor_size'],axis=1)
df_train2=df_train.drop(['tumor_size'],axis=1).drop_duplicates()
encode3 = dict(zip(df_train.key, df_train.tumor_size))
df_train2['tumor_size']=df_train2.key.map(encode3)
# df_train2.drop(['key'],axis=1,inplace=True)
# df_train.drop(['key'],axis=1,inplace=True)
#########################################################
df_train2=df_train2[df_train2.tumor_size>0]
######################################################
df_test2['is_what']='test'
df_train2['is_what']='train'
fulldf=df_train2.drop(['tumor_size'],axis=1).append(df_test2)
df_train2.drop(['is_what'],axis=1,inplace=True)
Y=df_train2['tumor_size']
#####feature enginneering######

# TEST=fulldf[fulldf['is_what']=='test'].drop(['is_what'],axis=1)
# TRAIN=fulldf[fulldf['is_what']=='train'].drop(['is_what','key'],axis=1)
# reduce_mem_usage(TRAIN)
# reduce_mem_usage(TEST)


stdml = dict(zip(df_train.key, df_train.std_dev_malign))
TRAIN['std_dev_malign']=TRAIN.key.map(stdml)
stdml2 = dict(zip(df_test.key, df_test.std_dev_malign))
TEST['std_dev_malign']=TEST.key.map(stdml2)

stdml3 = dict(zip(df_train.key, df_train.exposed_area))
TRAIN['exposed_area']=TRAIN.key.map(stdml3)
stdml4 = dict(zip(df_test.key, df_test.exposed_area))
TEST['exposed_area']=TEST.key.map(stdml4)

TRAIN = TRAIN[train_list]
TEST = TEST[train_list+["key"]]

df_test3 = df_test.copy()
test_key = TEST["key"]
df_test2 = TEST.copy()
df_train = TRAIN.copy()



In [6]:
df_test2

,size_npear / damage_size,PERCENTILE(damage_size) / damage_size,PERCENTILE(mass_npea) / PERCENTILE(exposed_area),malign_penalty / damage_size,PERCENTILE(exposed_area) / size_npear,damage_ratio * err_malign,malign_penalty / err_malign * malign_ratio / damage_size,damage_size / size_npear,malign_penalty / damage_size * malign_ratio / damage_size,malign_ratio / exposed_area * PERCENTILE(err_malign),malign_penalty / PERCENTILE(malign_penalty),damage_ratio / exposed_area * damage_size / malign_ratio,PERCENTILE(malign_penalty) / damage_size,malign_penalty / mass_npea * PERCENTILE(damage_ratio),err_malign / mass_npea * PERCENTILE(std_dev_malign),err_malign / damage_size * PERCENTILE(damage_ratio),damage_ratio * PERCENTILE(std_dev_malign),PERCENTILE(mass_npea) / PERCENTILE(malign_penalty),malign_ratio / err_malign * PERCENTILE(malign_penalty),PERCENTILE(std_dev_malign) / mass_npea,malign_penalty / std_dev_malign * malign_ratio / exposed_area,damage_size / mass_npea * std_dev_malign / mass_npea,PERCENTILE(damage_ratio) * PERCENTILE(err_malign),PERCENTILE(err_malign) / PERCENTILE(exposed_area),damage_ratio * PERCENTILE(err_malign),damage_ratio / damage_size * malign_penalty / damage_size,PERCENTILE(err_malign) / std_dev_malign,damage_ratio / std_dev_malign * damage_size,PERCENTILE(malign_penalty) / malign_ratio,PERCENTILE(std_dev_malign) / err_malign,damage_ratio * PERCENTILE(damage_size),err_malign / std_dev_malign * malign_ratio / mass_npea,PERCENTILE(malign_penalty) / err_malign,malign_penalty / size_npear * PERCENTILE(damage_ratio),err_malign / size_npear * malign_ratio / damage_ratio,err_malign / damage_size * malign_penalty / damage_size,mass_npea,size_npear,malign_ratio,damage_size,exposed_area,std_dev_malign,err_malign,malign_penalty,damage_ratio,key
0,16.754343,0.002750,1.202611,0.403251,0.000050,96571.261504,0.000032,0.059686,0.001366,1.645511e-08,143.831502,0.018688,0.002804,0.004035,0.031381,38.277490,2.829335,0.361877,0.000012,0.000014,9.147051e-08,0.000150,0.053550,1.282398,2.476369,0.310110,0.000880,34.136594,0.827733,0.000029,6.293637,0.001282,0.000066,0.021836,0.011091,17.013283,4946.44,914.06,0.18479,54.5566,6.628345e+05,67.0525,2301.76,22,41.9554,4946.44_914.06_0.18479_54.5566_662834.4535_67....
1,31.601245,0.002327,0.916535,0.931091,0.000069,131797.135000,0.000084,0.031644,0.005241,1.030272e-07,107.877551,0.009229,0.008631,0.007423,0.080710,54.475295,5.706152,0.231793,0.000040,0.000025,2.214701e-07,0.000131,0.241705,2.474288,11.343391,0.735826,0.003472,26.188691,1.533302,0.000043,4.886454,0.002085,0.000138,0.025578,0.014144,58.427559,5613.85,1629.12,0.29019,51.5524,7.842271e+05,80.1986,3235.00,48,40.7410,5613.85_1629.12_0.29019_51.5524_784227.1281_80...
2,19.865656,0.005250,0.985935,0.408677,0.000253,138092.586214,0.000022,0.050338,0.000654,1.065914e-07,109.966707,0.010451,0.003716,0.001086,0.287251,6.506784,22.451310,1.334369,0.000030,0.000061,4.713245e-08,0.000187,0.163260,0.983380,22.476240,0.078147,0.004081,24.318763,2.323764,0.000163,23.858576,0.000498,0.000122,0.004405,0.012794,12.419239,12421.10,3062.41,0.24654,154.1560,1.763575e+06,186.8590,4684.63,63,29.4778,12421.1_3062.41_0.24654_154.156_1763574.7673_1...
3,43.192998,0.003090,1.030000,0.738144,0.000093,123667.977211,0.000083,0.023152,0.004667,1.032668e-07,109.536917,0.006550,0.006739,0.004447,0.109195,40.822665,8.941203,0.615800,0.000043,0.000034,1.752768e-07,0.000113,0.192966,1.163981,10.273728,0.499486,0.002877,23.623857,1.065712,0.000072,6.770600,0.001824,0.000119,0.012359,0.012212,41.667598,6830.77,2457.66,0.35979,56.8995,9.296623e+05,92.7361,3211.93,42,38.5027,6830.77_2457.66_0.35979_56.8995_929662.2875_92...
4,56.390078,0.004387,1.000000,0.853593,0.000081,359436.030264,0.000024,0.017734,0.001441,8.016012e-08,196.687509,0.003461,0.004340,0.000677,0.400732,7.386908,26.841315,1.051664,0.000026,0.000030,3.049815e-08,0.000097,0.120414,0.996416,26.745123,0.104598,0.002019,11.920897,2.571114,0.000075,25.798731,0.000300,0.000071,0.001830,0.014930,52.169869,3

In [7]:
df_train

,size_npear / damage_size,PERCENTILE(damage_size) / damage_size,PERCENTILE(mass_npea) / PERCENTILE(exposed_area),malign_penalty / damage_size,PERCENTILE(exposed_area) / size_npear,damage_ratio * err_malign,malign_penalty / err_malign * malign_ratio / damage_size,damage_size / size_npear,malign_penalty / damage_size * malign_ratio / damage_size,malign_ratio / exposed_area * PERCENTILE(err_malign),malign_penalty / PERCENTILE(malign_penalty),damage_ratio / exposed_area * damage_size / malign_ratio,PERCENTILE(malign_penalty) / damage_size,malign_penalty / mass_npea * PERCENTILE(damage_ratio),err_malign / mass_npea * PERCENTILE(std_dev_malign),err_malign / damage_size * PERCENTILE(damage_ratio),damage_ratio * PERCENTILE(std_dev_malign),PERCENTILE(mass_npea) / PERCENTILE(malign_penalty),malign_ratio / err_malign * PERCENTILE(malign_penalty),PERCENTILE(std_dev_malign) / mass_npea,malign_penalty / std_dev_malign * malign_ratio / exposed_area,damage_size / mass_npea * std_dev_malign / mass_npea,PERCENTILE(damage_ratio) * PERCENTILE(err_malign),PERCENTILE(err_malign) / PERCENTILE(exposed_area),damage_ratio * PERCENTILE(err_malign),damage_ratio / damage_size * malign_penalty / damage_size,PERCENTILE(err_malign) / std_dev_malign,damage_ratio / std_dev_malign * damage_size,PERCENTILE(malign_penalty) / malign_ratio,PERCENTILE(std_dev_malign) / err_malign,damage_ratio * PERCENTILE(damage_size),err_malign / std_dev_malign * malign_ratio / mass_npea,PERCENTILE(malign_penalty) / err_malign,malign_penalty / size_npear * PERCENTILE(damage_ratio),err_malign / size_npear * malign_ratio / damage_ratio,err_malign / damage_size * malign_penalty / damage_size,mass_npea,size_npear,malign_ratio,damage_size,exposed_area,std_dev_malign,err_malign,malign_penalty,damage_ratio
0,56.319338,0.002361,0.890873,1.389162,0.000096,108590.311120,0.000212,0.017756,0.011288,5.840242e-08,112.779794,0.004899,0.012317,0.008144,0.149529,41.726166,14.786922,0.389618,0.000097,0.000054,2.800888e-07,0.000118,0.107493,0.491112,5.397371,1.054995,0.001252,18.633487,1.515843,0.000136,4.817423,0.001531,0.000231,0.019336,0.010112,73.941346,6930.90,2919.02,0.42116,51.8298,9.888294e+05,109.4870,2758.76,72,39.3620
1,21.826705,0.004311,1.037056,1.073585,0.000178,131106.259009,0.000056,0.045815,0.001499,1.458046e-07,245.039198,0.007665,0.004381,0.000467,0.340522,0.810120,19.700759,0.922126,0.000051,0.000057,1.461916e-07,0.000228,0.029261,1.104344,21.183132,0.105775,0.003864,19.783575,3.138612,0.000150,21.224799,0.000477,0.000165,0.001496,0.017284,28.586665,15635.70,4879.36,0.31206,223.5500,2.058426e+06,248.8810,5952.53,240,22.0253
2,20.527262,0.005688,1.006309,0.573282,0.000246,139040.649538,0.000031,0.048716,0.001134,1.311927e-07,113.035910,0.010569,0.005072,0.001659,0.300319,8.582887,20.165771,1.000379,0.000035,0.000065,8.006488e-08,0.000189,0.176170,1.163801,22.412272,0.135046,0.004667,23.858875,2.563663,0.000145,21.725303,0.000703,0.000139,0.006585,0.014892,20.868332,10376.20,2613.88,0.25191,127.3370,1.434676e+06,160.0930,4635.26,73,29.9963
3,29.014797,0.005233,1.003514,0.205867,0.000176,148382.441998,0.000013,0.034465,0.000456,1.682907e-07,121.143531,0.007006,0.001699,0.000390,0.291932,5.409737,20.392545,3.007794,0.000017,0.000055,3.515401e-08,0.000157,0.141176,1.118348,24.911767,0.037263,0.005118,25.277384,0.766895,0.000137,22.886978,0.000802,0.000050,0.001131,0.014316,6.984798,13093.80,4510.06,0.34444,155.4400,1.812195e+06,173.0150,5273.87,32,28.1354
4,33.860840,0.005605,0.999318,0.669614,0.000113,114282.517000,0.000078,0.029533,0.003005,1.061631e-07,108.814879,0.007475,0.006154,0.003630,0.210771,18.419143,17.066122,0.622238,0.000061,0.000065,1.676541e-07,0.000186,0.139387,0.889458,10.159705,0.275480,0.002332,23.960583,1.371235,0.000149,16.707590,0.001328,0.000161,0.009501,0.012350,25.670689,7545.21,2882.36,0.38201,85.1237,1.043918e+06,124.4140,3263.35,57,35.0200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [8]:
## SYANTAN's
df_train['Say_total_estimated_samples'] =   df_train['err_malign'] / df_train['std_dev_malign']  
df_train['Say_size_ratio_interaction'] = df_train['damage_size'] * df_train['damage_ratio']
df_train['Say_damaged_area'] = df_train['size_npear'] / df_train['damage_size']
df_train['Say_sorted_interaction'] = df_train['malign_penalty'] * df_train['damage_ratio']
df_train['Say_exposed_are_per_unit_mass'] = df_train['exposed_area'] / df_train['mass_npea']
df_train['Say_feat_1'] = df_train['malign_penalty'] * df_train['mass_npea']
df_train['Say_tumor_area'] = df_train['exposed_area'] - df_train['damage_size']
df_train['Say_skin_spread'] = df_train['Say_tumor_area'] / df_train['damage_ratio']
df_train['Say_tumor_mass'] = df_train['Say_tumor_area'] * df_train['mass_npea'] / df_train['size_npear']

#test
df_test2['Say_total_estimated_samples'] =   df_test2['err_malign'] / df_test2['std_dev_malign'] 
df_test2['Say_size_ratio_interaction'] = df_test2['damage_size'] * df_test2['damage_ratio']
df_test2['Say_damaged_area'] = df_test2['size_npear'] / df_test2['damage_size']
df_test2['Say_sorted_interaction'] = df_test2['malign_penalty'] * df_test2['damage_ratio']
df_test2['Say_exposed_are_per_unit_mass'] = df_test2['exposed_area'] / df_test2['mass_npea']
df_test2['Say_feat_1'] = df_test2['malign_penalty'] * df_test2['mass_npea']
df_test2['Say_tumor_area'] = df_test2['exposed_area'] - df_test2['damage_size']
df_test2['Say_skin_spread'] = df_test2['Say_tumor_area'] / df_test2['damage_ratio']
df_test2['Say_tumor_mass'] = df_test2['Say_tumor_area'] * df_test2['mass_npea'] / df_test2['size_npear']

In [11]:
# My Features
df_train["size_npear*malign_ratio/mass_npea"] = (df_train["size_npear"]*
                                       df_train["malign_ratio"])/df_train["mass_npea"]
df_test2["size_npear*malign_ratio/mass_npea"] = (df_test2["size_npear"]*
                                      df_test2["malign_ratio"])/df_test2["mass_npea"]
# #--------------------------------------------------
df_train["std_dev_malign_malign_ratio"] = (df_train["std_dev_malign"]*df_train["malign_ratio"])
df_test2["std_dev_malign_malign_ratio"] = (df_test2["std_dev_malign"]*df_test2["malign_ratio"])
# #--------------------------------------------------
df_train["exposed_area_by_damage_size"] = (df_train["exposed_area"]/df_train["damage_size"])
df_test2["exposed_area_by_damage_size"] = (df_test2["exposed_area"]/df_test2["damage_size"])
#--------------------------------------------------
df_train["Near_TumorRatio"] = df_train["damage_size"]/(df_train["std_dev_malign"]+(df_train["err_malign"]
                                                *df_train["malign_ratio"])/df_train["malign_penalty"])
df_test2["Near_TumorRatio"] = df_train["damage_size"]/(df_train["std_dev_malign"]+(df_test2["err_malign"]
                                                *df_test2["malign_ratio"])/df_train["malign_penalty"])
#--------------------------------------------------
df_train["exposed_area_by_size_npear"] = (df_train["exposed_area"]/df_train["size_npear"])*df_train["malign_ratio"]
df_test2["exposed_area_by_size_npear"] = (df_test2["exposed_area"]/df_test2["size_npear"])*df_test2["malign_ratio"]
#-------------------------------------------------
df_train["exposed_area_by_size_npear2"] = np.abs(df_train["exposed_area"]-df_train["size_npear"])/df_train["malign_ratio"]
df_test2["exposed_area_by_size_npear2"] = np.abs(df_test2["exposed_area"]-df_test2["size_npear"])/df_test2["malign_ratio"]
#-------------------------------------------------
df_train["Damaged_area"] = df_train["exposed_area"]*df_train["damage_ratio"]
df_test2["Damaged_area"] = df_test2["exposed_area"]*df_train["damage_ratio"]
#--------------------------------------------------
df_train["err_malign_by_malign_penalty"] = (df_train["err_malign"]/df_train["malign_penalty"])
df_test2["err_malign_by_malign_penalty"] = (df_test2["err_malign"]/df_test2["malign_penalty"])
#-------------------------------------------------
df_train["Density_npea"] = (df_train["mass_npea"]/df_train["size_npear"])
df_test2["Density_npea"] = (df_test2["mass_npea"]/df_test2["size_npear"])
#-----------------------NO USE!!!-------------------------
# df_train["Variance/std_dev_malign"] = (df_train["std_dev_malign"])**(1/2)/df_train["std_dev_malign"]
# df_test2["Variance/std_dev_malign"] = (df_test2["std_dev_malign"])**(1/2)/df_train["std_dev_malign"]
#---------------------------------------------------------
df_train["Malign_skin_size"] = (np.log(df_train["size_npear"])/(df_train["malign_ratio"]+1))+df_train["std_dev_malign"]-(df_train["err_malign"]/(df_train["malign_penalty"]+1))                                  
df_test2["Malign_skin_size"] = (np.log(df_test2["size_npear"])/(df_test2["malign_ratio"]+1))+df_test2["std_dev_malign"]-(df_test2["err_malign"]/(df_test2["malign_penalty"]+1))                                       
# #--------------------------------------------------------
df_train["Normal_skin_size"] = (df_train["size_npear"]*df_train["malign_ratio"])/(df_train["malign_ratio"]+1)
df_test2["Normal_skin_size"] = (df_test2["size_npear"]*df_test2["malign_ratio"])/(df_test2["malign_ratio"]+1)
# ----------------------------------------------------------
df_train["Maligin_skin_size2"] = (df_train["std_dev_malign"]*np.log(df_train.Malign_skin_size).std())+np.log(df_train.Malign_skin_size).mean()                                       
df_test2["Maligin_skin_size2"] = (df_test2["std_dev_malign"]*np.log(df_test2.Malign_skin_size).std())+np.log(df_test2.Malign_skin_size).mean()     

In [12]:
df_train

,size_npear / damage_size,PERCENTILE(damage_size) / damage_size,PERCENTILE(mass_npea) / PERCENTILE(exposed_area),malign_penalty / damage_size,PERCENTILE(exposed_area) / size_npear,damage_ratio * err_malign,malign_penalty / err_malign * malign_ratio / damage_size,damage_size / size_npear,malign_penalty / damage_size * malign_ratio / damage_size,malign_ratio / exposed_area * PERCENTILE(err_malign),malign_penalty / PERCENTILE(malign_penalty),damage_ratio / exposed_area * damage_size / malign_ratio,PERCENTILE(malign_penalty) / damage_size,malign_penalty / mass_npea * PERCENTILE(damage_ratio),err_malign / mass_npea * PERCENTILE(std_dev_malign),err_malign / damage_size * PERCENTILE(damage_ratio),damage_ratio * PERCENTILE(std_dev_malign),PERCENTILE(mass_npea) / PERCENTILE(malign_penalty),malign_ratio / err_malign * PERCENTILE(malign_penalty),PERCENTILE(std_dev_malign) / mass_npea,malign_penalty / std_dev_malign * malign_ratio / exposed_area,damage_size / mass_npea * std_dev_malign / mass_npea,PERCENTILE(damage_ratio) * PERCENTILE(err_malign),PERCENTILE(err_malign) / PERCENTILE(exposed_area),damage_ratio * PERCENTILE(err_malign),damage_ratio / damage_size * malign_penalty / damage_size,PERCENTILE(err_malign) / std_dev_malign,damage_ratio / std_dev_malign * damage_size,PERCENTILE(malign_penalty) / malign_ratio,PERCENTILE(std_dev_malign) / err_malign,damage_ratio * PERCENTILE(damage_size),err_malign / std_dev_malign * malign_ratio / mass_npea,PERCENTILE(malign_penalty) / err_malign,malign_penalty / size_npear * PERCENTILE(damage_ratio),err_malign / size_npear * malign_ratio / damage_ratio,err_malign / damage_size * malign_penalty / damage_size,mass_npea,size_npear,malign_ratio,damage_size,exposed_area,std_dev_malign,err_malign,malign_penalty,damage_ratio,Say_total_estimated_samples,Say_size_ratio_interaction,Say_damaged_area,Say_sorted_interaction,Say_exposed_are_per_unit_mass,Say_feat_1,Say_tumor_area,Say_skin_spread,Say_tumor_mass,size_npear*malign_ratio/mass_npea,std_dev_malign_malign_ratio,exposed_area_by_damage_size,Near_TumorRatio,exposed_area_by_size_npear,exposed_area_by_size_npear2,Damaged_area,err_malign_by_malign_penalty,Density_npea,Malign_skin_size,Normal_skin_size,Maligin_skin_size2
0,56.319338,0.002361,0.890873,1.389162,0.000096,108590.311120,0.000212,0.017756,0.011288,5.840242e-08,112.779794,0.004899,0.012317,0.008144,0.149529,41.726166,14.786922,0.389618,0.000097,0.000054,2.800888e-07,0.000118,0.107493,0.491112,5.397371,1.054995,0.001252,18.633487,1.515843,0.000136,4.817423,0.001531,0.000231,0.019336,0.010112,73.941346,6930.90,2919.02,0.42116,51.8298,9.888294e+05,109.4870,2758.76,72,39.3620,25.197147,2040.124588,56.319338,2834.0640,142.669698,499024.80,9.887776e+05,25120.105106,2.347746e+06,0.177376,46.111545,19078.395190,0.412578,142.669592,2.340940e+06,3.892230e+07,38.316111,2.374393,77.310197,865.050004,119.960364
1,21.826705,0.004311,1.037056,1.073585,0.000178,131106.259009,0.000056,0.045815,0.001499,1.458046e-07,245.039198,0.007665,0.004381,0.000467,0.340522,0.810120,19.700759,0.922126,0.000051,0.000057,1.461916e-07,0.000228,0.029261,1.104344,21.183132,0.105775,0.003864,19.783575,3.138612,0.000150,21.224799,0.000477,0.000165,0.001496,0.017284,28.586665,15635.70,4879.36,0.31206,223.5500,2.058426e+06,248.8810,5952.53,240,22.0253,23.917173,4923.755815,21.826705,5286.0720,131.649130,3752568.00,2.058203e+06,93447.206208,6.595423e+06,0.097383,77.665805,9207.901145,0.871130,131.646878,6.580616e+06,4.533746e+07,24.802208,3.204457,230.654557,1160.505679,267.251963
2,20.527262,0.005688,1.006309,0.573282,0.000246,139040.649538,0.000031,0.048716,0.001134,1.311927e-07,113.035910,0.010569,0.005072,0.001659,0.300319,8.582887,20.165771,1.000379,0.000035,0.000065,8.006488e-08,0.000189,0.176170,1.163801,22.412272,0.135046,0.004667,23.858875,2.563663,0.000145,21.725303,0.000703,0.000139,0.006585,0.014892,20.868332,10376.20,2613.88,0.25191,127.3370,1.434676e+06,160.0930,4635.26,73,29.9963,28.953546,3819.638853,20.527262,2189.7299,138.

In [ ]:
# Making Polunomial Features Works... but it consumes so much of time.... cant use with Catboost
# from sklearn.preprocessing import PolynomialFeatures,StandardScaler,MinMaxScaler
# df_train2 = pd.concat([df_train,df_test])
# df_train2 = df_train2.reset_index(drop=True)
# tumorsize = df_train2["tumor_size"].copy()
# trans = PolynomialFeatures(degree=3)
# data = trans.fit_transform(df_train2.drop("tumor_size",axis=1))
# df_train2 = pd.DataFrame(data)
# df_train2 = pd.concat([df_train2,tumorsize],axis=1)
# df_train2 = df_train2.drop(0,axis=1)
# df_train = df_train2[~df_train2.tumor_size.isnull()].copy()
# df_test = df_train2[df_train2.tumor_size.isnull()].drop("tumor_size",axis=1).reset_index(drop=True).copy()

In [ ]:
# feat = ETCCOLUMNS + ["exposed_area_by_damage_size"]

## Cross Validation

In [ ]:
# x = df_train.drop("tumor_size",axis=1)
# y = df_train["tumor_size"]
x = df_train
y = Y_TRAIN
#----------------------
folds = KFold(n_splits=20, shuffle=True, random_state=27)
oof_preds = []
feature_importance_df = pd.DataFrame()
feature_importance_df['Feature'] = x.columns
final_preds = []
for fold_, (trn_idx, val_idx) in enumerate(folds.split(x.values, y.values)):
        X_trn,y_trn = x[feat].iloc[trn_idx],y.iloc[trn_idx]
        X_val,y_val = x[feat].iloc[val_idx],y.iloc[val_idx]
        # clf=lgb.LGBMRegressor(random_state=27,n_estimators=1000)
        # clf.fit(X_trn, y_trn, eval_metric="rmse", eval_set=[(X_val,y_val)], verbose=0,
        #         early_stopping_rounds=200)
        clf=CatBoostRegressor(random_state=27,n_estimators=2000,task_type="GPU",
                              devices="0:1",boosting_type="Plain",
                              verbose=500,max_depth=13,learning_rate=0.09)
        clf.fit(X_trn, y_trn)
        #------------------importance-------------------
        # imp = importances(clf,X_val,y_val)
        # imp.rename(columns={
        #     'Importance':f'Importance_{fold_}'
        # },inplace=True)
        # feature_importance_df = pd.merge(feature_importance_df,imp,on='Feature')
        #-----------------------------------------------
        score = np.sqrt(mean_squared_error(y_pred=clf.predict(X_val),y_true=y_val))
        final_preds.append(score)
        oof_preds.append(clf.predict(df_test2[feat]))
        print(f"Fold {fold_+1}: {score} , Average {sum(final_preds)/(len(final_preds))}")
print(sum(final_preds)/(len(final_preds)))

0:	learn: 5.8633148	total: 101ms	remaining: 3m 21s
500:	learn: 0.5312739	total: 21.5s	remaining: 1m 4s
1000:	learn: 0.1462315	total: 42.9s	remaining: 42.8s
1500:	learn: 0.0526339	total: 1m 4s	remaining: 21.3s
1999:	learn: 0.0226938	total: 1m 24s	remaining: 0us
Fold 1: 3.698772914966341 , Average 3.698772914966341
0:	learn: 5.8671705	total: 92.2ms	remaining: 3m 4s
500:	learn: 0.5221891	total: 21.4s	remaining: 1m 4s
1000:	learn: 0.1350313	total: 42.8s	remaining: 42.7s
1500:	learn: 0.0478529	total: 1m 4s	remaining: 21.3s
1999:	learn: 0.0218358	total: 1m 25s	remaining: 0us
Fold 2: 3.5391942351404255 , Average 3.618983575053383
0:	learn: 5.8602630	total: 93.8ms	remaining: 3m 7s
500:	learn: 0.5528773	total: 21.4s	remaining: 1m 4s
1000:	learn: 0.1517515	total: 42.7s	remaining: 42.6s
1500:	learn: 0.0546224	total: 1m 3s	remaining: 21.2s
1999:	learn: 0.0239139	total: 1m 24s	remaining: 0us
Fold 3: 3.779205166089684 , Average 3.6723907720654836
0:	learn: 5.8531408	total: 94.9ms	remaining: 3m 9s
50

In [ ]:
final_preds = np.mean(oof_preds,0)

## Handling the Leak that we saved earlier

In [ ]:
##submission module oof
filen='CB_leak_with_36390162007397606.xlsx'
df_test2['tumor_size']=final_preds
encode2 = dict(zip(df_test2.key, df_test2.tumor_size))
df1=pd.DataFrame()
df1['tumor_size'] = df_test['tumor_size'].fillna(df_test['key'].map(encode2))
df1=df1.clip(lower=0)
df1.fillna(0,inplace=True)
###change download adress here according to ur drive####################

In [ ]:
display(Y_TRAIN.std())
# sub["tumor_size"].std()
df1.tumor_size.std()

tumor_size    6.081118
dtype: float64

4.564035843972792

In [ ]:
df1

,tumor_size
0,7.920434
1,10.160631
2,2.089449
3,11.530198
4,10.709364
...,...
36579,2.997622
36580,16.661810
36581,5.032029
36582,10.773860


In [ ]:
df1['tumor_size'].to_excel('CB_leak_with_36390162007397606.xlsx',index=False)

#### Addition to this  trying reg_resampler,Ensembling adds a bit of score... but i was out of my submission.. so only did 1 ensemble which is averaging my 20 fold (out of fold predictions) with 15 fold (out of fold predictions) this boosted a lot in my public LB score.. but in Final score it boosted a little.